# Load Initial

Since more data than can query at once, load manually

* Notably, if we use fixed cut offs if we are not aware of a pool
    * It will have missing data before the last checkpoint
    * Will need to additionally backfill new pools, 
        * and overwrite pool address list

In [ ]:
from datetime import datetime as dt
from datetime import timedelta, date

In [ ]:
from app.data.flipside_api_helper import query_and_save

In [ ]:
from app.curve.liquidity.fetch import generate_query, fetch
from app.curve.liquidity.fetch_cutoff import fetch_cutoff

In [ ]:
from app.data.local_storage import (
    pd,
    read_json,
    read_csv,
    write_dataframe_csv,
    write_dfs_to_xlsx,
    csv_to_df,
    df_to_csv,
    )
from app.utilities.utility import (
    utc,
    get_datetime_obj
    )
from app.data.reference import (
    filename_curve_liquidity, 
    filename_curve_liquidity_cutoff
    )

# PARAMS!

In [ ]:
load_initial = True
load_cutoff = True
load_forward = True

should_fetch = True 

cutoff_value = None

In [ ]:
def get_df_from_file(filename):
    resp_dict = read_csv(filename, 'raw_data')
    df = pd.json_normalize(resp_dict)
    return df

## Query cutoff

In [ ]:
if load_initial:
    if cutoff_value:
        df_cutoff = fetch_cutoff(cutoff_value)
    else:
        df_cutoff = fetch_cutoff()
    df_to_csv(df_cutoff, filename_curve_liquidity_cutoff, 'raw_data') 

In [ ]:
df_cutoff = get_df_from_file(filename_curve_liquidity_cutoff)
df_cutoff.head()

## Query back from current timestamp

In [ ]:
# df = query_and_save(query, filename, df):

In [ ]:
def backfiller(generate_query, filename, target, cutoff_time = None, page_size=10000):
    # Loop Basics
    should_continue = True
    df = get_df_from_file(filename)
    # Controls ending at a fixed cutoff
    if cutoff_time:
        cutoff_time = get_datetime_obj(cutoff_time)
    while should_continue:
        if len(df) == 0:
            df = []
            print("no starting position")
            time =  '2023-08-01 00:00:01'
            start_time = get_datetime_obj(time)
            end_time = start_time + timedelta(days=26*7)
        else:
            end_time = get_datetime_obj(df['block_timestamp'].min())
            start_time = end_time - timedelta(days=26*7)
            if cutoff_time and start_time < cutoff_time:
                start_time = cutoff_time
                print("Last Run!")
        # Begin true sequence
        print("="*50)
        print("="*50)
        print("GO BACK")
        print(f"Time Bounds: {start_time} --> {end_time}")
        # If things go off the rails, allow user to abort
        #   * Provides time to check in on api provider
        print("^^^ Look Up ^^^ User Input Requested ^^^")
        user_input = input("Type anything to abort, or enter to continue")
        if len(user_input) > 0:
            break
        current_length = len(df)
        query = generate_query(start_time, end_time)
        df = query_and_save(query, filename_curve_liquidity, df, page_size)
        if start_time == cutoff_time:
            should_continue = False
        if len(df) < current_length:
            should_continue = False

    
    return df


In [ ]:
if load_initial:
    df = backfiller(
        generate_query, 
        filename_curve_liquidity, 
        'block_timestamp', 
        df_cutoff.cutoff.min()
        )

In [ ]:
df_liquidity = get_df_from_file(filename_curve_liquidity)
df_liquidity.head()

# Query Forward

In [ ]:
if load_forward:
    if should_fetch:
        df_gauge_votes_raw = fetch(False)

# Join

In [ ]:
len(df) / 1000000

In [ ]:
def join_cutoff(df, df_cutoff):
    df_cutoff = format_cutoff(df_cutoff)
     
    return pd.concat([df, df_cutoff])

def format_cutoff(df_cutoff):
    cutoff = df_cutoff.cutoff.min()
    if not 'T' in cutoff:
        cutoff = cutoff.replace(' ', 'T')
    if not '.' in cutoff:
        cutoff = cutoff + '.000Z'
    if not 'Z' in cutoff:
        cutoff = cutoff + 'Z'
    df_cutoff['cutoff'] = cutoff
    df_cutoff = df_cutoff.rename(columns={
        "cutoff": 'block_timestamp',
        })   
    return df_cutoff

In [ ]:
combo_df = join_cutoff(df_liquidity, df_cutoff)
combo_df.block_timestamp.max()

In [ ]:
df_to_csv(combo_df, filename_curve_liquidity, 'source')

In [ ]:
combo_df['block_timestamp'] = pd.to_datetime(combo_df['block_timestamp'])

In [ ]:
# should_continue = True
# while should_continue:
#     user_input = input("Type anything to abort, or enter to continue")
#     if len(user_input) > 0:
#         print("Here!")
#         break

In [ ]:
dt.now(utc)
time =  '2023-08-01T00:00:01.000Z'
time = get_datetime_obj(time)
f"{time}"